In [8]:
import pandas as pd
import numpy as np
import pywikibot
from bs4 import BeautifulSoup, SoupStrainer
import requests
import re
from Constants import *
import pickle

### 1.Extract potential QIDS for a newspaper 

In [29]:
def get_QIDS(newspaper):
    url = f'https://www.wikidata.org/w/index.php?search={newspaper} \
    &title=Special%3ASearch&fulltext=1&ns0=1&ns120=1'
    page = requests.get(url)    
    data = page.text
    soup = BeautifulSoup(data)
    QIDS = []
    for link in soup.find_all('a'):
        link = link.get('href')
        if(link is not None and link.startswith("/wiki/Q")):
            QID =  link.replace('/wiki/','')
            QIDS.append(QID)
        
    return QIDS

In [36]:
df = pd.read_csv(FILE_NEWSPAPER_SPEAKER_COUNT,compression='bz2')

newspapers = df['newspaper'].unique()


array(['1011now', '101greatgoals', '1045theteam', ..., 'zerotackle',
       'zigwheels', 'zip06'], dtype=object)

In [37]:
print(len(newspapers))
newspapers

7362


array(['1011now', '101greatgoals', '1045theteam', ..., 'zerotackle',
       'zigwheels', 'zip06'], dtype=object)

In [ ]:
#Dictionnary of newspaper and found QIDS
newspapers_QIDS = {}

for i,newspaper in enumerate(newspapers):
    if(i%10==0):
        print(f'Iteration {i+1}')
    QIDS = get_QIDS(newspaper)
    print(newspaper,QIDS)
    newspapers_QIDS[newspaper]=QIDS

In [108]:
PICKLE_NEWSPAPER_QID = 'processed_data/newspaper_qids.pickle'
with open(PICKLE_NEWSPAPER_QID, 'wb') as handle:
    pickle.dump(newspapers_QIDS, handle, protocol=pickle.HIGHEST_PROTOCOL)

### 2.From potential QIDS, get correct QID as well as owner name if exists

In [70]:
owned_by_key = 'P127'
official_website_key = 'P856'
site = pywikibot.Site('en', 'wikipedia')
repo = site.data_repository()

def extract_website_owner(newspaper,QIDS):
    current_QID = None
    owner = None
    official_website = None
    
    for QID in QIDS:
        current_QID = QID
        item = pywikibot.ItemPage(repo, QID)
        if(item.exists()):
            if(item.isRedirectPage()):
                item = page.getRedirectTarget()
                
            claims = item.get()['claims'].toJSON()# Provides dict["property_key"->property]
            
            official_website_property = claims.get(official_website_key)
            if(official_website_property!=None):
                official_website = official_website_property[0]['mainsnak']['datavalue']['value']
                
                if(newspaper in official_website):
                    owned_by_property = claims.get(owned_by_key)#Return None if property not found
                    if(owned_by_property!=None):
                        owner_id = owned_by_property[0]['mainsnak']['datavalue']['value']['numeric-id']
                        #Use ItemPage to obtain a dictionnary
                        data_found = pywikibot.ItemPage(repo, "Q" + str(owner_id))
                        #Get title in english
                        owner = data_found.get()['labels']['en']
                    break
                    
    return current_QID, official_website, owner

In [65]:
newspapers_QIDS = None
with open(PICKLE_NEWSPAPER_QID, 'rb') as handle:
    newspapers_QIDS = pickle.load(handle)

print(len(newspapers_QIDS))

7362


In [ ]:
newspapers_info = {}

for i, (newspaper,QIDS) in enumerate(newspapers_QIDS.items()):
    if(i%10==0):
        print(f'Iteration {i+1}')
    infos= extract_website_owner(newspaper,QIDS)
    newspapers_info[newspaper] = infos
    print(newspaper,infos)

In [115]:
missing = set(newspapers_QIDS.keys()).difference(set(newspapers_info.keys()))

In [116]:
for m in missing:
    newspapers_info[m] = (None,None,None)

In [118]:
NEWSPAPER_INFO_PICKLE ='processed_data/newspaper_infos.pickle'
with open(NEWSPAPER_INFO_PICKLE, 'wb') as handle:
    pickle.dump(newspapers_info, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [123]:
flattened = [[k,v1,v2,v3]for k,(v1,v2,v3) in newspapers_info.items()]

In [131]:
df = pd.DataFrame(flattened,columns = ['newspaper','QID','website','owner'])

In [129]:
df

,newspaper,QID,website,owner
0,1011now,Q6334998,https://www.1011now.com/,None
1,101greatgoals,None,None,None
2,1045theteam,Q7956286,http://www.1045theteam.com,Townsquare Media
3,1070thefan,Q7949418,http://www.1070thefan.com/,Emmis Communications
4,107jamz,Q6331512,http://www.107jamz.com,Townsquare Media
...,...,...,...,...
7357,zip06,Q27929737,https://www.7-zip.org/,None
7358,networkworld,None,None,None
7359,rnews,None,None,None
7360,mamamia,None,None,None


In [135]:
df.to_csv('processed_data/newspaper-owner.csv',index=False)

### 3.For newspapers with QID, extract geographical information if available

In [177]:
repo = site.data_repository()
site = pywikibot.Site('en', 'wikipedia')

def get_property_QID_value(property_dic):
    ''' Get QID from property whose value is a QID and get label corresponding to this QID'''
    
    QID =  f"Q{property_dic[0]['mainsnak']['datavalue']['value']['numeric-id']}"
    page = pywikibot.ItemPage(repo, QID)
    property_value = page.get()['labels']['en']
    
    return QID,property_value

def get_property_value(property_dic):
    ''' Get value from property whose value is its label'''
    
    property_value = property_dic[0]['mainsnak']['datavalue']['value']
    
    return property_value

def extract_geographical_properties(QID):
    
    country_key = 'P17'
    region_key = 'P131'
    coordinates_key = 'P625'
    headquarters_location_key= 'P159'
    

    country = ''
    region = ''
    latitude = np.nan
    longitude = np.nan
    
    item = pywikibot.ItemPage(repo, QID)
    claims = item.get()['claims'].toJSON()# Provides dict["property_key"->property]        

    country_property = claims.get(country_key)
    region_property = claims.get(region_key)
    coordinates_property = claims.get(coordinates_key)
    headquarters_location_property = claims.get(headquarters_location_key)
    
    if(country_property!=None):
        
        QID,country = get_property_QID_value(country_property)
        
    if(headquarters_location_property!=None):
        QID,region = get_property_QID_value(headquarters_location_property)
        
        try:
            coordinates_full = headquarters_location_property[0]['qualifiers'][coordinates_key][0]['datavalue']['value']
            latitude = coordinates_full['latitude']
            longitude = coordinates_full['longitude'] 
            
        except:
            pass
        
        
    else:
        if(region_property!=None):
        
            QID,region = get_property_QID_value(region_property)
    
        if(coordinates_property!=None):
            coordinates_full = get_property_value(coordinates_property)
            latitude = coordinates_full['latitude']
            longitude = coordinates_full['longitude']  

    return country, region, latitude, longitude

In [178]:
print(extract_geographical_properties('Q7956286'))
print(extract_geographical_properties('Q6334998'))

('United States of America', 'New York', 42.879, -73.863)
('United States of America', 'Lincoln', 40.80305556, -97.18111111)


In [176]:
df = pd.read_csv('processed_data/newspaper-owner.csv')

df = df.fillna('')
df

,newspaper,QID,website,owner
0,1011now,Q6334998,https://www.1011now.com/,
1,101greatgoals,,,
2,1045theteam,Q7956286,http://www.1045theteam.com,Townsquare Media
3,1070thefan,Q7949418,http://www.1070thefan.com/,Emmis Communications
4,107jamz,Q6331512,http://www.107jamz.com,Townsquare Media
...,...,...,...,...
7357,zip06,Q27929737,https://www.7-zip.org/,
7358,networkworld,,,
7359,rnews,,,
7360,mamamia,,,


In [179]:
geographical_infos = []
for idx,row in df.iterrows():
    country = ''
    region = ''
    latitude = np.nan
    longitude = np.nan
    
    if(row['QID']!=''):
        try:
            country, region, latitude, longitude = extract_geographical_properties(row['QID'])
        except:
            pass
        
    geographical_infos.append([country, region, latitude, longitude])
    
    print(f"{row['newspaper']}:{country,region,latitude,longitude}")
    
    if(idx%10==1):
        print(f'Iteration {idx}')
    
geographical_info

1011now:('United States of America', 'Lincoln', 40.80305556, -97.18111111)
101greatgoals:('', '', nan, nan)
Iteration 1
1045theteam:('United States of America', 'New York', 42.879, -73.863)
1070thefan:('United States of America', 'Indiana', 39.9558, -86.3583)
107jamz:('United States of America', 'Louisiana', 30.202, -92.947)
10news:('United States of America', 'San Diego', 32.83888889, -117.24888889)
1130thetiger:('United States of America', 'Louisiana', 32.705, -93.8819)
1380kcim:('United States of America', 'Iowa', 42.0414, -94.885)
13abc:('United States of America', 'Toledo', 41.651824, -83.642371)
1428elm:('', '', nan, nan)
14850:('', '', nan, nan)
14news:('United States of America', '', 37.88730556, -87.51880556)
Iteration 11
15rounds:('', '', nan, nan)
1888pressrelease:('', '', nan, nan)
220triathlon:('', '', nan, nan)
225batonrouge:('', '', nan, nan)
247sports:('', '', nan, nan)
247wallst:('', '', nan, nan)
24hourhiphop:('United States of America', '', nan, nan)
2dayfm:('', '', 

al:('Brazil', 'Brazil', -9.57, -36.55)
al-monitor:('United States of America', '', nan, nan)
Iteration 181
alabamanews:('United States of America', 'Montgomery', 32.14175, -86.74505556)
alarabiya:('', '', nan, nan)
alaraby:('', '', nan, nan)
alaskahighwaynews:('Canada', '', nan, nan)
albawaba:('Jordan', 'Amman', nan, nan)
albernivalleynews:('', '', nan, nan)
albionnewsonline:('', '', nan, nan)
aldergrovestar:('', 'Aldergrove, British Columbia', nan, nan)
alderleyedge:('United Kingdom', '', nan, nan)
aldianews:('', '', nan, nan)
Iteration 191
aledotimesrecord:('', '', nan, nan)
aleteia:('', '', nan, nan)
alextimes:('', '', nan, nan)
algemeiner:('United States of America', 'New York City', nan, nan)
alibi:('United Kingdom', '', nan, nan)
aliran:('', '', nan, nan)
aljazeera:('Qatar', 'Doha', 25.2819, 51.5175)
all-noise:('', '', nan, nan)
all4women:('', '', nan, nan)
allaboutalpha:('', '', nan, nan)
Iteration 201
allaboutjazz:('', 'Philadelphia', nan, nan)
allaboutthejersey:('', '', nan, n

astrobio:('', '', nan, nan)
astrobiology:('United States of America', 'Ames Research Center', nan, nan)
astronomy:('', '', nan, nan)
Iteration 371
astronomynow:('', '', nan, nan)
asu:('United States of America', 'Tempe', nan, nan)
asumag:('', '', nan, nan)
athleticsnation:('', '', nan, nan)
atholdailynews:('United States of America', 'Athol', nan, nan)
atimes:("People's Republic of China", 'Hong Kong', nan, nan)
atlantablackstar:('United States of America', 'Atlanta', nan, nan)
atlantadailyworld:('United States of America', 'Atlanta', nan, nan)
atlantafalcons:('United States of America', 'Flowery Branch', nan, nan)
atlantamagazine:('', '', nan, nan)
Iteration 381
atlanticcouncil:('United States of America', 'Washington, D.C.', nan, nan)
atlanticsentinel:('', '', nan, nan)
atlasobscura:('United States of America', 'Greenpoint', nan, nan)
atmmarketplace:('', '', nan, nan)
atpworldtour:('', '', nan, nan)
attackofthefanboy:('', '', nan, nan)
atthehive:('', '', nan, nan)
attitude:('United K

belfasttelegraph:('United Kingdom', 'Belfast', nan, nan)
beliefnet:('', '', nan, nan)
bellanaija:('Nigeria', 'Lagos', nan, nan)
bellingencourier:('', '', nan, nan)
belmarrahealth:('', '', nan, nan)
beloitdailynews:('United States of America', 'Beloit', nan, nan)
belpernews:('', '', nan, nan)
bemidjipioneer:('United States of America', '', nan, nan)
Iteration 571
benarnews:('', '', nan, nan)
bendigoadvertiser:('', '', nan, nan)
bendsource:('', '', nan, nan)
benefitscanada:('', '', nan, nan)
benefitspro:('', '', nan, nan)
bengals:('United States of America', 'Cincinnati', nan, nan)
benningtonbanner:('United States of America', 'Bennington', nan, nan)
bentoneveningnews:('', '', nan, nan)
benzinga:('', '', nan, nan)
berkeley:('United States of America', 'Berkeley', nan, nan)
Iteration 581
berkeleydailyplanet:('United States of America', 'Berkeley', nan, nan)
berkeleyside:('', 'Berkeley', nan, nan)
berkshireeagle:('United States of America', 'Pittsfield', nan, nan)
bernama:('Malaysia', 'Kua

blogto:('', '', nan, nan)
bloodhorse:('', '', nan, nan)
bloodhorse:443:('', '', nan, nan)
bloody-disgusting:('', '', nan, nan)
bloodyelbow:('', '', nan, nan)
Iteration 701
blooloop:('', '', nan, nan)
bloomberg:('United States of America', 'Bloomberg Tower', nan, nan)
blr:('Germany', 'Munich', nan, nan)
blu-ray:('', '', nan, nan)
blueandgreentomorrow:('', '', nan, nan)
bluebirdbanter:('', '', nan, nan)
bluegrasstoday:('', '', nan, nan)
bluemountainsgazette:('', '', nan, nan)
blueshirtbanter:('', '', nan, nan)
bmmagazine:('', '', nan, nan)
Iteration 711
bnn:('Netherlands', 'Hilversum', nan, nan)
bnnbloomberg:('Canada', '299 Queen Street West', nan, nan)
boatingindustry:('', '', nan, nan)
bobsguide:('', '', nan, nan)
bobvila:('', '', nan, nan)
bocamag:('', '', nan, nan)
bocanewsnow:('', '', nan, nan)
bocaratontribune:('United States of America', '', nan, nan)
bodyandsoul:('', '', nan, nan)
bodyshopbusiness:('', '', nan, nan)
Iteration 721
boereport:('', '', nan, nan)
bognor:('United Kingd

...

villagemagazine:('', 'Dublin', nan, nan)
villagesoup:('', '', nan, nan)
villagevoice:('United States of America', 'New York City', 40.7283, -73.9911)
vin:('Vietnam', 'Nghệ An', 18.673333333333, 105.69222222222)
Iteration 6791
vindy:('United States of America', 'Youngstown', nan, nan)
vinepair:('', '', nan, nan)
vinography:('', '', nan, nan)
virginia:('United States of America', 'United States of America', 37.5, -79)
virginiabusiness:('', '', nan, nan)
virtual-strategy:('', '', nan, nan)
virtueonline:('', '', nan, nan)
visualcapitalist:('', '', nan, nan)
vitalfootball:('', '', nan, nan)
vivaelbirdos:('', '', nan, nan)
Iteration 6801
vivathematadors:('', '', nan, nan)
vmikeydets:('United States of America', '', nan, nan)
vnews:('United States of America', 'Port Arthur', 29.921725, -93.9138687)
voanews:('United States of America', 'Washington, D.C.', nan, nan)
vocm:('Canada', "St. John's", 47.547658333333, -52.783044444444)
vogue:('', 'Times Square', nan, nan)
voice-online:('', '', nan, n

weny:('United States of America', 'Elmira', nan, nan)
wesa:('', '', nan, nan)
Iteration 6951
wesleyan:('United States of America', 'Middletown', 41.55562, -72.655844)
wesleyanargus:('', '', nan, nan)
wesm913:('United States of America', 'Maryland', 38.2103, -75.6822)
westcoastsentinel:('', '', nan, nan)
westender:('', '', nan, nan)
westernadvocate:('Australia', 'Bathurst', -33.416998, 149.577675)
westernbulldogs:('Australia', 'Melbourne', nan, nan)
westerninvestor:('', '', nan, nan)
westernjournal:('United States of America', 'Phoenix', nan, nan)
westerntelegraph:('United Kingdom', '', nan, nan)
Iteration 6961
westfaironline:('United States of America', 'Fairfield', nan, nan)
westfargopioneer:('United States of America', 'West Fargo', nan, nan)
westhawaiitoday:('United States of America', 'Kailua-Kona', nan, nan)
westleedsdispatch:('United Kingdom', 'Leeds', 53.797208, -1.662129)
westlondonsport:('', '', nan, nan)
westmeathexaminer:('', '', nan, nan)
westmeathindependent:('', '', nan, 

wksu:('United States of America', 'Ohio', 41.0828, -81.6339)
wktv:('', '', 43.10233333, -74.93861111)
wkyt:('United States of America', 'Lexington', 38.03972222, -84.40277778)
wkzo:('United States of America', 'Michigan', 42.3486, -85.5633)
wlfi:('United States of America', 'Indiana', 40.38888889, -86.61277778)
wlni:('United States of America', 'Virginia', 37.4269, -79.1239)
wlox:('United States of America', 'Biloxi', 30.72277778, -89.09111111)
wlrn:('United States of America', 'Florida', 25.97997222, -80.19597222)
wltribune:('', '', nan, nan)
Iteration 7121
wltz:('', 'Columbus', 32.45788889, -84.88561111)
wm:('South Korea', 'Seoul', nan, nan)
wmagazine:('', '', nan, nan)
wmbfnews:('United States of America', '', 33.73080556, -79.07538889)
wmcactionnews5:('United States of America', '', 35.16916667, -89.88611111)
wmdt:('United States of America', '', 38.50166667, -75.73572222)
wmfe:('United States of America', '', nan, nan)
wmich:('United States of America', 'Kalamazoo', 42.283178, -85

wwnytv:('United States of America', '', nan, nan)
www:('', '', nan, nan)
wwwdiatoday:('', '', nan, nan)
wwwicbookmovie:('', '', nan, nan)
Iteration 7271
wwwicmix:('', '', nan, nan)
wwwicsbeat:('', '', nan, nan)
wwwmdiginews:('', '', nan, nan)
wwwmunityadvocate:('', '', nan, nan)
wwwnadianunderwriter:('', '', nan, nan)
wwwplex:('', '', nan, nan)
wwwputerweekly:('', '', nan, nan)
wwwputerworld:('', '', nan, nan)
wwwroparlropa:('', '', nan, nan)
wwwropolropa:('', '', nan, nan)
Iteration 7281
wxow:('', '', nan, nan)
wxpr:('United States of America', 'Wisconsin', 45.774, -89.248)
wxxinews:('', '', nan, nan)
wxyz:('United States of America', 'Detroit', 42.471521, -83.246925)
wycombewanderers:('United Kingdom', 'High Wycombe', nan, nan)
wydaily:('', '', nan, nan)
wymondhamandattleboroughmercury:('', '', nan, nan)
wymt:('', '', nan, nan)
wyofile:('', '', nan, nan)
wyomingpublicmedia:('United States of America', 'Wyoming', 44.8206, -108.852)
Iteration 7291
wypr:('United States of America', 'Mar

[['United States of America', 'Lincoln', 40.80305556, -97.18111111],
 ['nan', 'nan', nan, nan],
 ['United States of America', 'New York', 42.879, -73.863],
 ['United States of America', 'Indiana', 39.9558, -86.3583],
 ['United States of America', 'Louisiana', 30.202, -92.947],
 ['United States of America', 'San Diego', 32.83888889, -117.24888889],
 ['United States of America', 'Louisiana', 32.705, -93.8819]]

In [181]:
len(geographical_infos)

7362

In [182]:
with open('processed_data/newspaper_geo_info.pickle', 'wb') as handle:
    pickle.dump(geographical_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [204]:
geographical_array = np.array(geographical_infos)

df = df.assign(country=geographical_array[:,0],region = geographical_array[:,1],
               latitude = geographical_array[:,2], longitude = geographical_array[:,3])

df.to_csv('processed_data/newspaper-infos.csv',index=False)

In [202]:
df[(df['owner']!='') & (df['country']!='')].groupby('country').size().sort_values(ascending=False)

country
United States of America                       491
United Kingdom                                  95
Canada                                          71
Australia                                       22
India                                            8
New Zealand                                      5
France                                           5
Ireland                                          4
South Africa                                     4
Singapore                                        4
Germany                                          3
Turkey                                           3
Philippines                                      3
Pakistan                                         3
Nigeria                                          2
Israel                                           2
United Kingdom of Great Britain and Ireland      2
Ghana                                            2
People's Republic of China                       2
Japan                  

### Exract wikipedia URL of data from QID

In [18]:
df = pd.read_csv('processed_data/newspaper-infos.csv')
df = df.fillna('')
df

,newspaper,QID,website,owner,country,region,latitude,longitude
0,1011now,Q6334998,https://www.1011now.com/,,United States of America,Lincoln,40.803056,-97.181111
1,101greatgoals,,,,,,,
2,1045theteam,Q7956286,http://www.1045theteam.com,Townsquare Media,United States of America,New York,42.879,-73.863
3,1070thefan,Q7949418,http://www.1070thefan.com/,Emmis Communications,United States of America,Indiana,39.9558,-86.3583
4,107jamz,Q6331512,http://www.107jamz.com,Townsquare Media,United States of America,Louisiana,30.202,-92.947
...,...,...,...,...,...,...,...,...
7357,zip06,Q27929737,https://www.7-zip.org/,,,,,
7358,networkworld,,,,,,,
7359,rnews,,,,,,,
7360,mamamia,,,,,,,


In [56]:
site = pywikibot.Site('en', 'wikipedia')
repo = site.data_repository()
def get_wikipedia_url(QID):
    wikipedia_URL = ''
    item = pywikibot.ItemPage(repo, QID)
    links = item.get()['sitelinks'].toJSON()
    if(len(links)!=0):
        title = links['enwiki']['title']
        title = title.replace(' ','_')
        wikipedia_URL = f'https://en.wikipedia.org/wiki/{title}'
    
    return wikipedia_URL

In [57]:
wikipedia_urls = []
for idx,row in df.iterrows():
    wikipedia_url = ''
    
    if(row['QID']!=''):
        try:
            wikipedia_url = get_wikipedia_url(row['QID'])
        except Exception as e:
            print(e)
        
    wikipedia_urls.append(wikipedia_url)
    
    print(f"{row['newspaper'],row['QID']}:{wikipedia_url}")
    
    if(idx%10==1):
        print(f'Iteration {idx}')
    
wikipedia_urls

('1011now', 'Q6334998'):https://en.wikipedia.org/wiki/KOLN
('101greatgoals', ''):
Iteration 1
('1045theteam', 'Q7956286'):https://en.wikipedia.org/wiki/WTMM-FM
('1070thefan', 'Q7949418'):https://en.wikipedia.org/wiki/WFNI
('107jamz', 'Q6331512'):https://en.wikipedia.org/wiki/KJMH
('10news', 'Q3191396'):https://en.wikipedia.org/wiki/KGTV
('1130thetiger', 'Q6340648'):https://en.wikipedia.org/wiki/KWKH
('1380kcim', 'Q6326840'):https://en.wikipedia.org/wiki/KCIM
('13abc', 'Q2386816'):https://en.wikipedia.org/wiki/WTVG
('1428elm', 'Q27335792'):
('14850', 'Q23533712'):
('14news', 'Q7949314'):https://en.wikipedia.org/wiki/WFIE
Iteration 11
('15rounds', 'Q46017852'):
('1888pressrelease', ''):
('220triathlon', ''):
('225batonrouge', ''):
('247sports', 'Q16973497'):https://en.wikipedia.org/wiki/247Sports.com
('247wallst', 'Q68896999'):
('24hourhiphop', 'Q72292189'):
('2dayfm', 'Q4633461'):https://en.wikipedia.org/wiki/2day_FM_(Fiji)
('2oceansvibe', ''):
('2paragraphs', 'Q51049229'):
Iteration 21

('aim', 'Q2003140'):https://en.wikipedia.org/wiki/Aim_(musician)
('ainonline', 'Q48797965'):https://en.wikipedia.org/wiki/Aviation_International_News
('aip', 'Q4651980'):https://en.wikipedia.org/wiki/AIP_Conference_Proceedings
('airalamo', ''):
('airdrieecho', ''):
('airforcetimes', 'Q4697970'):https://en.wikipedia.org/wiki/Air_Force_Times
('airlinesanddestinations', ''):
('airport-business', 'Q2739991'):https://en.wikipedia.org/wiki/Northeast_Philadelphia_Airport
('airport-world', 'Q29257'):https://en.wikipedia.org/wiki/TWA_Flight_Center
Iteration 171
('airspacemag', 'Q4697697'):https://en.wikipedia.org/wiki/Air_&_Space/Smithsonian
('aish', 'Q2828774'):https://en.wikipedia.org/wiki/Aish_HaTorah
('ajc', 'Q3092348'):https://en.wikipedia.org/wiki/The_Atlanta_Journal-Constitution
('ajot', 'Q104069164'):
('ajournalofmusicalthings', ''):
('ajp', 'Q26853994'):
('akihabaranews', ''):
('akron', 'Q163132'):https://en.wikipedia.org/wiki/Akron,_Ohio
('al', 'Q40885'):https://en.wikipedia.org/wiki/

('armidaleexpress', 'Q19870768'):https://en.wikipedia.org/wiki/Armidale_Express
Iteration 321
('army', 'Q222595'):https://en.wikipedia.org/wiki/British_Army
('army-technology', 'Q56172742'):
('armytimes', 'Q689778'):https://en.wikipedia.org/wiki/Army_Times
('arnnet', ''):
('arrl', 'Q466576'):https://en.wikipedia.org/wiki/American_Radio_Relay_League
('arrowheadaddict', ''):
('arrowheadpride', ''):
('arsenal-world', 'Q8870090'):https://en.wikipedia.org/wiki/Category:United_States_Army_arsenals_during_World_War_II
('arsenalinsider', ''):
('arstechnica', 'Q584152'):https://en.wikipedia.org/wiki/Ars_Technica
Iteration 331
('artesianews', 'Q4797565'):https://en.wikipedia.org/wiki/Artesia_Daily_Press
('artfixdaily', 'Q46489297'):
('artforum', 'Q386599'):https://en.wikipedia.org/wiki/Artforum
('artlyst', 'Q769724'):https://en.wikipedia.org/wiki/Eli_Lotar
('artnet', 'Q266566'):https://en.wikipedia.org/wiki/Artnet
('artnews', 'Q705816'):https://en.wikipedia.org/wiki/ARTnews
('artshub', ''):
('ar

('baltic-course', 'Q40088924'):https://en.wikipedia.org/wiki/The_Baltic_Course
('baltimorebeatdown', ''):
('baltimorepostexaminer', 'Q55667227'):
('baltimoreravens', 'Q276539'):https://en.wikipedia.org/wiki/Baltimore_Ravens
('bamsmackpow', ''):
Iteration 481
('banana1015', 'Q7957122'):https://en.wikipedia.org/wiki/WWBN
('banbridgeleader', ''):
('banburycake', 'Q4854006'):https://en.wikipedia.org/wiki/Banbury_Cake_(newspaper)
('banburyguardian', 'Q4854009'):https://en.wikipedia.org/wiki/Banbury_Guardian
('bancroftthisweek', 'Q4854167'):https://en.wikipedia.org/wiki/Bancroft_This_Week
('bandt', 'Q4678672'):https://en.wikipedia.org/wiki/Adam_Bandt
('bangaloremirror', 'Q978676'):https://en.wikipedia.org/wiki/Bangalore_Mirror
('bangordailynews', 'Q4855555'):https://en.wikipedia.org/wiki/Bangor_Daily_News
('bankerandtradesman', 'Q86826270'):
('bankinfosecurity', 'Q11199928'):
Iteration 491
('bankingday', ''):
('bankofcanada', 'Q806703'):https://en.wikipedia.org/wiki/Bank_of_Canada
('bankrate

('bizwest', 'Q39045358'):https://en.wikipedia.org/wiki/BizWest
('bjconline', ''):
('bjpenn', 'Q402779'):https://en.wikipedia.org/wiki/B.J._Penn
('bkreader', ''):
('blabbermouth', 'Q182638'):https://en.wikipedia.org/wiki/Blabbermouth.net
('blackagendareport', ''):
Iteration 661
('blackandteal', ''):
('blackburnnews', ''):
('blackenterprise', 'Q4920731'):https://en.wikipedia.org/wiki/Black_Enterprise
('blackhawkup', ''):
('blackheartgoldpants', ''):
('blackhillsfox', 'Q23017069'):https://en.wikipedia.org/wiki/KEVN-LD
('blacknews', ''):
('blackpoolgazette', 'Q4923252'):https://en.wikipedia.org/wiki/Blackpool_Gazette
('blackshoediaries', ''):
('blackstarnews', 'Q105576469'):https://en.wikipedia.org/wiki/Black_Star_News
Iteration 671
('blacktownsun', ''):
('blackwhitereadallover', ''):
'enwiki'
('blastingnews', 'Q24951763'):
('blastmagazine', ''):
('blastr', 'Q27172365'):
('blayneychronicle', ''):
('blazersedge', ''):
('bleacherreport', 'Q2906353'):https://en.wikipedia.org/wiki/Bleacher_Rep

...

('voice-online', 'Q7225106'):https://en.wikipedia.org/wiki/Political_repression_of_cyber-dissidents
('voiceofalexandria', 'Q6341506'):https://en.wikipedia.org/wiki/KXRA_(AM)
('voiceofdetroit', ''):
('voiceofoc', 'Q7939562'):https://en.wikipedia.org/wiki/Voice_of_OC
Iteration 6811
('voiceofsandiego', 'Q13447154'):https://en.wikipedia.org/wiki/Voice_of_San_Diego
('voiceofsikkim', ''):
('voiceofthevalley', ''):
('voiceonline', ''):
('volanteonline', ''):
('voltairenet', 'Q2629867'):https://en.wikipedia.org/wiki/Voltaire_Network
('vooks', 'Q92507767'):
('vox', 'Q15630787'):https://en.wikipedia.org/wiki/Vox_(political_party)
('voxy', 'Q17146630'):https://en.wikipedia.org/wiki/Voxy
('vpr', 'Q7921754'):https://en.wikipedia.org/wiki/Vermont_Public_Radio
Iteration 6821
('vrfocus', ''):
('vtdigger', 'Q57983508'):https://en.wikipedia.org/wiki/VTDigger
('vuhoops', ''):
('vulcanadvocate', ''):
('vulture', 'Q27881700'):
('wa', 'Q1223'):https://en.wikipedia.org/wiki/Washington_(state)
('waaytv', 'Q33

('westfaironline', 'Q55665894'):
('westfargopioneer', 'Q55667969'):
('westhawaiitoday', 'Q7985331'):https://en.wikipedia.org/wiki/West_Hawaii_Today
'enwiki'
('westleedsdispatch', 'Q26599915'):
('westlondonsport', ''):
('westmeathexaminer', 'Q7989086'):https://en.wikipedia.org/wiki/Westmeath_Examiner
('westmeathindependent', ''):
('westport-news', 'Q58249774'):
('westportnow', 'Q55665951'):
('westseattleblog', ''):
Iteration 6971
('westsenecabee', 'Q55668407'):
('westsideseattle', ''):
('westsidetoday', 'Q55665599'):
('westsussextoday', ''):
('westword', 'Q7989927'):https://en.wikipedia.org/wiki/Westword
('westyorkshire', 'Q104604325'):
('wetherbynews', 'Q7990083'):https://en.wikipedia.org/wiki/Wetherby_News
('wetinhappen', ''):
('wetzelchronicle', ''):
('wexfordpeople', 'Q7990255'):https://en.wikipedia.org/wiki/Wexford_People
Iteration 6981
('weyburnreview', ''):
('wfae', 'Q7949161'):https://en.wikipedia.org/wiki/WFAE
('wfdd', 'Q7949237'):https://en.wikipedia.org/wiki/WFDD
('wfirnews',

('wltz', 'Q7952518'):https://en.wikipedia.org/wiki/WLTZ
('wm', 'Q19794482'):https://en.wikipedia.org/wiki/WM_Entertainment
('wmagazine', 'Q1853094'):https://en.wikipedia.org/wiki/W_(magazine)
('wmbfnews', 'Q7952661'):https://en.wikipedia.org/wiki/WMBF-TV
('wmcactionnews5', 'Q7952679'):https://en.wikipedia.org/wiki/WMC-TV
('wmdt', 'Q7952728'):https://en.wikipedia.org/wiki/WMDT
('wmfe', 'Q55803618'):
('wmich', 'Q1546191'):https://en.wikipedia.org/wiki/Western_Michigan_University
('wmnf', 'Q7952954'):https://en.wikipedia.org/wiki/WMNF
('wmot', 'Q101208396'):
Iteration 7131
('wmra', 'Q7953023'):https://en.wikipedia.org/wiki/WMRA
('wnax', 'Q14709794'):https://en.wikipedia.org/wiki/WNAX_(AM)
('wnba', 'Q2593221'):https://en.wikipedia.org/wiki/Women's_National_Basketball_Association
('wnbf', 'Q16903576'):https://en.wikipedia.org/wiki/WNBF
('wncw', 'Q7953305'):https://en.wikipedia.org/wiki/WNCW
('wnd', 'Q3569943'):https://en.wikipedia.org/wiki/WorldNetDaily
('wndu', 'Q7953321'):https://en.wikip

('wxow', 'Q7957821'):https://en.wikipedia.org/wiki/WXOW
('wxpr', 'Q7957828'):https://en.wikipedia.org/wiki/WXPR
('wxxinews', ''):
('wxyz', 'Q3564870'):https://en.wikipedia.org/wiki/WXYZ-TV
('wycombewanderers', 'Q48953'):https://en.wikipedia.org/wiki/Wycombe_Wanderers_F.C.
('wydaily', 'Q91079038'):
('wymondhamandattleboroughmercury', ''):
('wymt', 'Q7958085'):https://en.wikipedia.org/wiki/WYMT-TV
('wyofile', ''):
('wyomingpublicmedia', 'Q6339821'):https://en.wikipedia.org/wiki/KUWP
Iteration 7291
('wypr', 'Q2538121'):https://en.wikipedia.org/wiki/WYPR
('wyrk', 'Q7958139'):https://en.wikipedia.org/wiki/WYRK
('wyso', 'Q7958165'):https://en.wikipedia.org/wiki/WYSO
('wzzm13', 'Q7958441'):https://en.wikipedia.org/wiki/WZZM
('x17online', 'Q564426'):https://en.wikipedia.org/wiki/Annals_of_Mathematics
('xcmag', ''):
('xconomy', 'Q8043395'):https://en.wikipedia.org/wiki/Xconomy
('xda-developers', 'Q262900'):https://en.wikipedia.org/wiki/XDA_Developers
'enwiki'
('xinhuanet', 'Q11081577'):
('xxlma

['https://en.wikipedia.org/wiki/KOLN',
 '',
 'https://en.wikipedia.org/wiki/WTMM-FM',
 'https://en.wikipedia.org/wiki/WFNI',
 'https://en.wikipedia.org/wiki/KJMH',
 'https://en.wikipedia.org/wiki/KGTV',
 'https://en.wikipedia.org/wiki/KWKH',
 'https://en.wikipedia.org/wiki/KCIM',
 'https://en.wikipedia.org/wiki/WTVG',
 '',
 '',
 'https://en.wikipedia.org/wiki/WFIE',
 '',
 '',
 '',
 '',
 'https://en.wikipedia.org/wiki/247Sports.com',
 '',
 '',
 'https://en.wikipedia.org/wiki/2day_FM_(Fiji)',
 '',
 '',
 '',
 'https://en.wikipedia.org/wiki/List_of_Yugoslav_World_War_II_monuments_and_memorials_in_Croatia',
 'https://en.wikipedia.org/wiki/3AW',
 '',
 '',
 '',
 'https://en.wikipedia.org/wiki/WMGT-TV',
 'https://en.wikipedia.org/wiki/Avengers:_Endgame',
 'https://en.wikipedia.org/wiki/San_Francisco_49ers',
 'https://en.wikipedia.org/wiki/4BC',
 '',
 'https://en.wikipedia.org/wiki/5280',
 'https://en.wikipedia.org/wiki/CKGL',
 '',
 'https://en.wikipedia.org/wiki/CKTB',
 'https://en.wikipedia.o